In [16]:
import json
import os
import time
import glob
import sys
import numpy as np
from tqdm import tqdm
from skimage.measure import label, regionprops,regionprops_table
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import functions as fnc
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [17]:
grainsize=pd.read_csv('/DATA/vito/data/sedinet/SediNet/grain_size_global/global_all4.csv')

In [ ]:
gravelnsize=pd.read_csv('/DATA/vito/data/sedinet/SediNet/grain_size_gravel_generic/gravel_all4.csv')
sandsize=pd.read_csv('/DATA/vito/data/sedinet/SediNet/grain_size_sand_generic/sand_all3.csv')
sievedsandsize=pd.read_csv('/DATA/vito/data/sedinet/SediNet/grain_size_sieved_sands/data_pescadero_sieve.csv')

In [18]:
grainsize['P5_p']=np.zeros(len(grainsize))
grainsize['P10_p']=np.zeros(len(grainsize))
grainsize['P16_p']=np.zeros(len(grainsize))
grainsize['P25_p']=np.zeros(len(grainsize))
grainsize['P50_p']=np.zeros(len(grainsize))
grainsize['P75_p']=np.zeros(len(grainsize))
grainsize['P84_p']=np.zeros(len(grainsize))
grainsize['P90_p']=np.zeros(len(grainsize))
grainsize['P95_p']=np.zeros(len(grainsize))

In [19]:
fn_img = glob.glob('/DATA/vito/data/'+'sedinet/SediNet/images/*')
fn_img.sort()


In [20]:
id=0
file_name=fn_img[id].split("/")[-2]+'/'+fn_img[id].split("/")[-1]
int(np.where(np.array(grainsize['files'])==file_name)[0][0])

191

In [58]:
from tifffile import imread
def load_image(DataDIR,DSname,fid):
    fn_img = glob.glob(DataDIR+DSname)
    fn_img.sort()
    if fn_img[fid][-3:]=='npy':
        #image=(np.load(fn_img[fid])*255).astype(np.uint8)
        image=(np.load(fn_img[fid])).astype(np.uint8)
    elif fn_img[fid][-3:]=='tif':
        image = imread(fn_img[fid])
    else:
        image = cv2.imread(fn_img[fid])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print(fn_img[fid].split("/")[-1]+' imported')
    return image

In [59]:
id_list=np.array(range(378))
id_list=id_list[id_list!=382]
sedi_ids=[]
percentile=[5,10,16,25,50,75,84,90,95]
var='axis_minor_length'
for id in id_list:
    file_name=fn_img[id].split("/")[-2]+'/'+fn_img[id].split("/")[-1]
    where=np.where(np.array(grainsize['files'])==file_name)[0]
    if len(where)!=0:
        OutDIR=f'/DATA/vito/output/Sedinet/sedinet_{id}_up2_org/'
        with open(OutDIR+'init_para.json', 'r') as json_file:
            init_para = json.load(json_file)[0]
        DataDIR=init_para.get('DataDIR')
        DSname=init_para.get('DatasetName')
        fid=init_para.get('fid')
        image=load_image(DataDIR,DSname,fid)
        mask_path_1 = OutDIR + 'Merged/all_mask_merged_windows_id_001.npy'
        mask_path_0 = OutDIR + 'Merged/all_mask_merged_windows_id_000.npy'

        if os.path.exists(mask_path_1):
            masks = np.array(np.load(mask_path_1, allow_pickle=True))
        elif os.path.exists(mask_path_0):
            masks = np.array(np.load(mask_path_0, allow_pickle=True))
        else:
            continue
        #masks=fnc.clean_and_overwrite(masks)
        masks=cv2.resize(label(masks).astype(np.uint16), (0, 0), fx = 1/2, fy = 1/2, interpolation = cv2.INTER_NEAREST)
        
        labeled = label(masks, background=0)
        props = regionprops_table(
            labeled,
            properties=('label', var),
        )
        props_df = pd.DataFrame(props)
        area=np.array(props_df[var])

    
        sedi_id = int(where[0])
        sedi_ids.append(sedi_id)
        for p in percentile:
            grainsize.loc[sedi_id,f'P{p}_p']=np.percentile(area,p)*2
    
    

0017_cu_2.jpg imported
0024_makro.JPG imported
0026_makro.jpg imported
0028_makro.JPG imported
0029_1_Bali.JPG imported
0041_cu_1.jpg imported


libpng warning: iCCP: known incorrect sRGB profile


0053_Makro.png imported
0055_cu_2.jpg imported
0062_cu_1.jpg imported
0070_cu_1.jpg imported
0079_cu_2.jpg imported
0094_cu_1.jpg imported
0095_cu_2.jpg imported
0124_cu_1.jpg imported
0132-makro.jpg imported
0133_cu_2.jpg imported
0151_cu_2.jpg imported
0177_cu_1.jpg imported
0177_cu_2.jpg imported
0185_cu_2.jpg imported
0210_cu_1.jpg imported
0242_cu_2.jpg imported
0247_cu_1.jpg imported
0271_cu_1.jpg imported
0271_cu_2.jpg imported
0286_cu_2.jpg imported
0315_cu_2.jpg imported
0330.jpg imported
0354_cu_2.jpg imported
0413_cu_1.jpg imported
0434_cu_1.jpg imported
0434_cu_2.jpg imported
0453_cu_2.jpg imported
0463_cu_1.jpg imported
0463_cu_2.jpg imported
047020139.tiff imported
047080139.tiff imported
047140138.tiff imported
047200139.tiff imported
0477.jpg imported
048020138.tiff imported
048080139.tiff imported
048140139.tiff imported
048200139.tiff imported
0483_cu_1.jpg imported
049020138.tiff imported
049080139.tiff imported
049140138.tiff imported
049200139.tiff imported
0551_cu

In [61]:
pred=grainsize.loc[sedi_ids]
for p in percentile:
    mean_diff=np.mean(pred[f'P{p}']-pred[f'P{p}_p'])
    print(f'Mean P{p} difference: {mean_diff:.4f}/ {mean_diff/np.mean(pred[f'P{p}'])*100:.2f}%')
    

Mean P5 difference: 5.8489/ 36.15%
Mean P10 difference: 9.7028/ 40.68%
Mean P16 difference: 17.1011/ 50.09%
Mean P25 difference: 27.3422/ 57.03%
Mean P50 difference: 67.9269/ 68.43%
Mean P75 difference: 131.2670/ 73.38%
Mean P84 difference: 161.4932/ 73.66%
Mean P90 difference: 207.1355/ 75.26%
Mean P95 difference: 250.1248/ 74.94%


In [62]:
id_list=np.array(range(378))
id_list=id_list[id_list!=382]
sedi_ids=[]
percentile=[5,10,16,25,50,75,84,90,95]
var='axis_major_length'
for id in id_list:
    file_name=fn_img[id].split("/")[-2]+'/'+fn_img[id].split("/")[-1]
    where=np.where(np.array(grainsize['files'])==file_name)[0]
    if len(where)!=0:
        OutDIR=f'/DATA/vito/output/Sedinet/sedinet_{id}_up2_org/'
        with open(OutDIR+'init_para.json', 'r') as json_file:
            init_para = json.load(json_file)[0]
        DataDIR=init_para.get('DataDIR')
        DSname=init_para.get('DatasetName')
        fid=init_para.get('fid')
        image=load_image(DataDIR,DSname,fid)
        mask_path_1 = OutDIR + 'Merged/all_mask_merged_windows_id_001.npy'
        mask_path_0 = OutDIR + 'Merged/all_mask_merged_windows_id_000.npy'

        if os.path.exists(mask_path_1):
            masks = np.array(np.load(mask_path_1, allow_pickle=True))
        elif os.path.exists(mask_path_0):
            masks = np.array(np.load(mask_path_0, allow_pickle=True))
        else:
            continue
        #masks=fnc.clean_and_overwrite(masks)
        masks=cv2.resize(label(masks).astype(np.uint16), (0, 0), fx = 1/2, fy = 1/2, interpolation = cv2.INTER_NEAREST)
        
        labeled = label(masks, background=0)
        props = regionprops_table(
            labeled,
            properties=('label', var),
        )
        props_df = pd.DataFrame(props)
        area=np.array(props_df[var])

    
        sedi_id = int(where[0])
        sedi_ids.append(sedi_id)
        for p in percentile:
            grainsize.loc[sedi_id,f'P{p}_p']=np.percentile(area,p)*2

0017_cu_2.jpg imported
0024_makro.JPG imported
0026_makro.jpg imported
0028_makro.JPG imported
0029_1_Bali.JPG imported
0041_cu_1.jpg imported


libpng warning: iCCP: known incorrect sRGB profile


0053_Makro.png imported
0055_cu_2.jpg imported
0062_cu_1.jpg imported
0070_cu_1.jpg imported
0079_cu_2.jpg imported
0094_cu_1.jpg imported
0095_cu_2.jpg imported
0124_cu_1.jpg imported
0132-makro.jpg imported
0133_cu_2.jpg imported
0151_cu_2.jpg imported
0177_cu_1.jpg imported
0177_cu_2.jpg imported
0185_cu_2.jpg imported
0210_cu_1.jpg imported
0242_cu_2.jpg imported
0247_cu_1.jpg imported
0271_cu_1.jpg imported
0271_cu_2.jpg imported
0286_cu_2.jpg imported
0315_cu_2.jpg imported
0330.jpg imported
0354_cu_2.jpg imported
0413_cu_1.jpg imported
0434_cu_1.jpg imported
0434_cu_2.jpg imported
0453_cu_2.jpg imported
0463_cu_1.jpg imported
0463_cu_2.jpg imported
047020139.tiff imported
047080139.tiff imported
047140138.tiff imported
047200139.tiff imported
0477.jpg imported
048020138.tiff imported
048080139.tiff imported
048140139.tiff imported
048200139.tiff imported
0483_cu_1.jpg imported
049020138.tiff imported
049080139.tiff imported
049140138.tiff imported
049200139.tiff imported
0551_cu

In [65]:
pred=grainsize.loc[sedi_ids]
for p in percentile:
    mean_diff=np.mean(pred[f'P{p}']-pred[f'P{p}_p'])
    print(f'Mean P{p} difference: {mean_diff:.4f}/ {mean_diff/np.mean(pred[f'P{p}'])*100:.2f}%')

Mean P5 difference: -1.4310/ -8.85%
Mean P10 difference: 0.6128/ 2.57%
Mean P16 difference: 6.1788/ 18.10%
Mean P25 difference: 13.8075/ 28.80%
Mean P50 difference: 47.9511/ 48.30%
Mean P75 difference: 102.9649/ 57.56%
Mean P84 difference: 128.8017/ 58.75%
Mean P90 difference: 169.9129/ 61.74%
Mean P95 difference: 205.9461/ 61.70%


In [14]:
np.percentile(props_df['area'],50)

np.float64(6208.0)

In [3]:
df=fnc.get_props_df(image, masks, init_para.get('resample_factor'),res=init_para.get('resolution(mm)') )

In [5]:
import dataframe_image as dfi
df_styled = df.style.background_gradient().format(precision=2)
dfi.export(df_styled, 'fig/ac_table_b100.png',table_conversion = 'matplotlib')

ValueError: Your Styled DataFrame has more than 100 rows and will produce a huge image file, possibly causing your computer to crash. Override this error by explicitly setting `max_rows` to -1 for all columns. Styled DataFrames are unable to select a subset of rows or columns and therefore do not work with the `max_rows` and `max_cols` parameters